## Logistic Regression 


In this notebook we will train a [Logistic Regression model](https://en.wikipedia.org/wiki/Logistic_regression) to distinguish between legitimate and fraudulent transactions. 

Logistic Regression is a classic statistical technique used for binary classification. Here the binary variable we are predicting is 'legitimate' or 'not legitimate' (i.e. fraudulent).

We begin by loading in our generated data.

In [1]:
import numpy as np
import pandas as pd
df = pd.read_parquet("fraud-cleaned-sample.parquet")

We need to split our data set into training and testing sets:

In [2]:
from sklearn import model_selection
train, test = model_selection.train_test_split(df, random_state=43)

We also load in the feature engineering pipeline stage which we developed in [notebook 2](02-feature-engineering.ipynb). The model takes the feature vectors as input, rather than the raw data.

In [3]:
import cloudpickle as cp
feature_pipeline = cp.load(open('feat_pipeline.pkl', 'rb'))

In [4]:
feature_pipeline

Pipeline(steps=[('feature_extraction',
                 ColumnTransformer(transformers=[('interarrival_scaler',
                                                  Pipeline(steps=[('median_imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('interarrival_scaler',
                                                                   RobustScaler())]),
                                                  ['interarrival']),
                                                 ('amount_scaler',
                                                  RobustScaler(), ['amount']),
                                                 ('onehot',
                                                  OneHotEncoder(categories=[['online',
                                                                             'contactless',
                                                                       

#### Dealing with Imbalanced Classes

When the training data set contains unequal representation from each of your classes we say we are dealing with 'imbalanced classes'. In our data set fewer than 2% of the samples are fraudulent, and the remaining 98% are legitimate. Thus we have imbalanced classes. 

This causes problems for a few reasons:
1. A model which classifies all transactions as 'legitimate' would be correct 98% of the time. This high accuracy can trick you into thinking that your model is working well, despite it just returning 'legitimate' for every sample it sees. 
2. Even if your model tries to learn patterns in the data, it may struggle to learn from the 'fraudulent' data since there simply isn't enough of it.


There are a few approaches we could take to tackle the problem, and today we will use two of them: 
1. We will use metrics which are more informative than simply counting how often the model makes a correct prediction. 
2. We will weight the samples by the inverse of the frequency of their label within the data set. These weights will be passed into the logistic regression model, and used to ensure that the model is penalised proportionally to this weight for making a misclassification for each class when it is training. 


In this next cell we compute these weights for each of the data labels. 

In [5]:
import warnings
warnings.filterwarnings('ignore')

In [6]:
fraud_frequency = train[train["label"] == "fraud"]["timestamp"].count() / train["timestamp"].count()
train.loc[train["label"] == "legitimate", "weights"] = fraud_frequency
train.loc[train["label"] == "fraud", "weights"] = (1 - fraud_frequency)


We're now ready to train our Logistic Regression model. The model is trained on the feature vectors (generated using our `feature_pipeline` from the previous notebook) and we pass the class weights we computed above as a model parameter. 

In [7]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(max_iter=500)

svecs = feature_pipeline.fit_transform(train)
lr.fit(svecs, train["label"], sample_weight=train["weights"])

LogisticRegression(max_iter=500)

We need to validate our model to check how well it performs on data it wasn't trained on. We use the model we just trained to make predictions for the data in our test set, and compare those predictions to the truth. 



In [8]:
from sklearn.metrics import classification_report

predictions = lr.predict(feature_pipeline.fit_transform(test))
print(classification_report(test.label.values, predictions))


              precision    recall  f1-score   support

       fraud       0.15      0.92      0.26     11424
  legitimate       1.00      0.90      0.95    613576

    accuracy                           0.90    625000
   macro avg       0.58      0.91      0.60    625000
weighted avg       0.98      0.90      0.94    625000



The report shows that the model is performing okay, but is much better at identifying legitimate transactions than fraudulent ones. 

We can visualise the accuracy of classifications in a binary confusion matrix.

In [9]:
from mlworkflows import plot
df, chart = plot.binary_confusion_matrix(test["label"], predictions)
chart

alt.Chart(...)

Viewing the raw counts, as well as the proportions of correctly and incorrectly classified items, emphasises that the model often misclassifies 'fraudulent' transactions as 'legitimate'. 

In [10]:
df

,predicted,actual,raw_count,value
0,fraud,fraud,10539,0.922532
1,fraud,legitimate,885,0.077468
2,legitimate,fraud,58896,0.095988
3,legitimate,legitimate,554680,0.904012


We want to save the model so that we can use it outside of this notebook. 

In [11]:
cp.dump(lr, open("lr.pkl", "wb"))
